In [1]:
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

import pandas as pd
import numpy as np
import sys
import h5py
import pickle
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from scipy.sparse import csr_matrix, hstack
from sklearn.cross_validation import KFold, train_test_split
from keras.layers.advanced_activations import PReLU
from keras.models import Sequential
from keras.models import save_model, load_model
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
import matplotlib.patches as mpatches


%matplotlib inline

/home/coder.chenshicheng/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/coder.chenshicheng/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
ids = test['id']
test['loss'] = np.nan
joined = pd.concat([train, test])
f_num = [f for f in train.columns if 'cont' in f]
f_cat = [f for f in train.columns if 'cat' in f]
for column in f_cat:
    if (train[column].nunique() != test[column].nunique()
        or (train[column].unique() != test[column].unique()).any()):
        set_train = set(train[column].unique())
        set_test = set(test[column].unique())
        
        remove = ((set_train - set_test)|(set_test - set_train))
        def filter_cat(x):
            if x in remove:
                return np.nan
            return x
        myfilter = lambda x: np.nan if x in remove else x
        joined[column] = joined[column].apply(filter_cat, 1)

    #joined[column] = pd.factorize(joined[column].values, sort=True)[0]

train = joined[joined['loss'].notnull()]
test = joined[joined['loss'].isnull()]

shift = 200
#train_y = np.log(train['loss'] + shift)
train_y=train['loss']
train_x = train.drop(['loss', 'id'], 1)
test = test.drop(['loss', 'id'], 1)

In [3]:
scaler = StandardScaler()
contx = scaler.fit_transform(train_x[f_num])
catx = pd.get_dummies(data=train_x[f_cat],columns=f_cat,dummy_na=False).values

print (catx.shape,contx.shape)

train_x = np.concatenate((contx,catx),axis=1)


test = pd.get_dummies(data=test,columns=f_cat).values
print (train_x.shape)
print (test.shape)
print (train_y.shape)

(188318, 1065) (188318, 14)
(188318, 1079)
(125546, 1079)
(188318,)


In [4]:
import keras.backend as K

def mae_score(y_true, y_pred):
    return K.mean(K.abs(K.exp(y_true)-K.exp(y_pred)), axis=-1)

from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', 
                             verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

In [5]:
# http://cs231n.github.io/neural-networks-3/

import numpy as np
import scipy.sparse as sp

from keras import backend as K
from keras.callbacks import Callback
from keras.models import load_model

import sys
import warnings
class ExponentialMovingAverage(Callback):
    """create a copy of trainable weights which gets updated at every
       batch using exponential weight decay. The moving average weights along
       with the other states of original model(except original model trainable
       weights) will be saved at every epoch if save_mv_ave_model is True.
       If both save_mv_ave_model and save_best_only are True, the latest
       best moving average model according to the quantity monitored
       will not be overwritten. Of course, save_best_only can be True
       only if there is a validation set.
       This is equivalent to save_best_only mode of ModelCheckpoint
       callback with similar code. custom_objects is a dictionary
       holding name and Class implementation for custom layers.
       At end of every batch, the update is as follows:
       mv_weight -= (1 - decay) * (mv_weight - weight)
       where weight and mv_weight is the ordinal model weight and the moving
       averaged weight respectively. At the end of the training, the moving
       averaged weights are transferred to the original model.
       """
    def __init__(self, decay=0.999, filepath='model3/model{epoch:02d}-{val_loss:.2f}.hdf5',
                 save_mv_ave_model=False, verbose=0,
                 save_best_only=False, monitor='val_loss', mode='auto',
                 save_weights_only=False, custom_objects={}):
        self.decay = decay
        self.filepath = filepath
        self.verbose = verbose
        self.save_mv_ave_model = save_mv_ave_model
        self.save_weights_only = save_weights_only
        self.save_best_only = save_best_only
        self.monitor = monitor
        self.custom_objects = custom_objects  # dictionary of custom layers
        self.sym_trainable_weights = None  # trainable weights of model
        self.mv_trainable_weights_vals = None  # moving averaged values
        self.epochs = 0

        if mode not in ['auto', 'min', 'max']:
            warnings.warn('ModelCheckpoint mode %s is unknown, '
                          'fallback to auto mode.' % (mode),
                          RuntimeWarning)
            mode = 'auto'

        if mode == 'min':
            self.monitor_op = np.less
            self.best = np.Inf
        elif mode == 'max':
            self.monitor_op = np.greater
            self.best = -np.Inf
        else:
            if 'acc' in self.monitor:
                self.monitor_op = np.greater
                self.best = -np.Inf
            else:
                self.monitor_op = np.less
                self.best = np.Inf

    def on_train_begin(self, logs={}):
        self.sym_trainable_weights = self.model.trainable_weights
        # Initialize moving averaged weights using original model values
        self.mv_trainable_weights_vals = {x.name: K.get_value(x).copy() for x in
                                          self.sym_trainable_weights}
        if self.verbose:
            print('Created a copy of model weights to initialize moving averaged weights.')
    def on_train_end(self, logs={}):
        '''old = K.get_value(self.model.trainable_weights[0])
        name = self.model.trainable_weights[0].name
        assert((old != self.mv_trainable_weights_vals[name]).any())'''
        
        for weight in self.sym_trainable_weights:
            K.set_value(weight, self.mv_trainable_weights_vals[weight.name])

        '''old = K.get_value(self.model.trainable_weights[0])
        name = self.model.trainable_weights[0].name
        assert((old == self.mv_trainable_weights_vals[name]).all())'''
            
    def on_batch_end(self, batch, logs={}):
        if(self.epochs + 5 > self.params['epochs']):
            for weight in self.sym_trainable_weights:
                old_val = self.mv_trainable_weights_vals[weight.name].copy()
                self.mv_trainable_weights_vals[weight.name] = \
                    old_val * self.decay + (1.0 - self.decay) * K.get_value(weight).copy()
            #assert((old_val == self.mv_trainable_weights_vals[weight.name]).all())
            #assert((old_val == K.get_value(weight)).all())

    def on_epoch_end(self, epoch, logs={}):
        self.epochs += 1
        if(self.epochs + 5 > self.params['epochs']):
            filepath = self.filepath.format(epoch=epoch + 1, **logs)
            self.model.save(filepath, overwrite=True)

In [6]:
from keras import regularizers
from keras.initializers import he_normal
from keras.regularizers import l2
from keras.activations import selu
from keras.optimizers import Adam
def hyper_model(seed = None):
    model = Sequential()
    model.add(Dense(437, input_dim=train_x.shape[1], kernel_initializer=he_normal(seed = seed)
                    ,kernel_regularizer=l2(0.002)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.536))
    
    model.add(Dense(182, kernel_initializer=he_normal(seed = seed),kernel_regularizer=l2(0.002)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    
    model.add(Dense(73, kernel_initializer=he_normal(seed = seed),kernel_regularizer=l2(0.002)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.233))
    
    model.add(Dense(1, kernel_initializer=he_normal(seed = seed),kernel_regularizer=l2(0.002)))
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    #model.compile(optimizer='adadelta',loss = 'mae',metrics = [mae_score])
    model.compile(optimizer='adadelta',loss = 'mae')
    return model

In [7]:
nnmodel={}
nnmodel.clear()
def cross_validate_mlp(mlp_func, nfolds=10,nbags=5):
    folds = KFold(len(train_y), n_folds=nfolds, shuffle=True, random_state = 31337)
    val_scores = np.zeros((nbags,))
    stack_train = np.zeros((nbags,len(train_y)))
    stack_test = np.zeros((nbags,len(test)))
    for k,(train_index, test_index) in enumerate(folds):
        xtr = train_x[train_index]
        ytr = train_y[train_index]
        xte = train_x[test_index]
        yte = train_y[test_index]
        for bag in range(nbags):
            nnmodel['nn%d',k*10+bag*1] = mlp_func(seed = k*10+bag*1)
            early_stopping = EarlyStopping(monitor='val_loss', patience=5)
            fit = nnmodel['nn%d',k*10+bag*1].fit(xtr, ytr, validation_split=0.2, batch_size=128,
                          epochs=30, verbose=1, callbacks=[ExponentialMovingAverage()])
            pred = nnmodel['nn%d',k*10+bag*1].predict(xte, batch_size=1024)
            #score = mean_absolute_error(np.exp(yte), np.exp(pred))
            score = mean_absolute_error(yte, pred)
            val_scores[bag] += score
            #stack_train[bag][test_index] = pred[:,0]
            print ("nfold:{},bag:{}".format(k,bag),score)
    for bag in range(nbags):
        val_scores[bag] = val_scores[bag] / float(nfolds)
    
    
    return val_scores

cv_score = cross_validate_mlp(hyper_model)
print ("CV score for the final model:", cv_score)

Train on 135588 samples, validate on 33898 samples
Epoch 1/30
135588/135588 [==============================] - 14s 100us/step - loss: 1925.6660 - val_loss: 1233.0826
Epoch 2/30
135588/135588 [==============================] - 13s 95us/step - loss: 1472.5587 - val_loss: 1212.4064
Epoch 3/30
135588/135588 [==============================] - 13s 94us/step - loss: 1459.4909 - val_loss: 1210.9034
Epoch 4/30
135588/135588 [==============================] - 13s 94us/step - loss: 1448.2593 - val_loss: 1196.8310
Epoch 5/30
135588/135588 [==============================] - 13s 94us/step - loss: 1437.9991 - val_loss: 1185.3718
Epoch 6/30
135588/135588 [==============================] - 12s 85us/step - loss: 1426.9940 - val_loss: 1181.3583
Epoch 7/30
135588/135588 [==============================] - 13s 93us/step - loss: 1418.1283 - val_loss: 1181.7783
Epoch 8/30
135588/135588 [==============================] - 13s 93us/step - loss: 1405.6848 - val_loss: 1179.7277
Epoch 9/30
135588/135588 [==========

135588/135588 [==============================] - 13s 96us/step - loss: 1289.3202 - val_loss: 1162.8053
Epoch 11/30
135588/135588 [==============================] - 13s 95us/step - loss: 1285.5039 - val_loss: 1165.0546
Epoch 12/30
135588/135588 [==============================] - 13s 98us/step - loss: 1285.1570 - val_loss: 1161.5478
Epoch 13/30
135588/135588 [==============================] - 12s 91us/step - loss: 1281.0012 - val_loss: 1160.1952
Epoch 14/30
135588/135588 [==============================] - 13s 96us/step - loss: 1278.9509 - val_loss: 1164.2880
Epoch 15/30
135588/135588 [==============================] - 13s 97us/step - loss: 1278.6519 - val_loss: 1161.2380
Epoch 16/30
135588/135588 [==============================] - 13s 97us/step - loss: 1276.1171 - val_loss: 1158.7853
Epoch 17/30
135588/135588 [==============================] - 13s 94us/step - loss: 1274.5203 - val_loss: 1157.8897
Epoch 18/30
135588/135588 [==============================] - 13s 94us/step - loss: 1268.1803

135588/135588 [==============================] - 11s 84us/step - loss: 1293.4744 - val_loss: 1154.2033
Epoch 21/30
135588/135588 [==============================] - 12s 85us/step - loss: 1291.3564 - val_loss: 1156.6765
Epoch 22/30
135588/135588 [==============================] - 12s 86us/step - loss: 1286.6473 - val_loss: 1155.3640
Epoch 23/30
135588/135588 [==============================] - 12s 85us/step - loss: 1283.2246 - val_loss: 1154.5849
Epoch 24/30
135588/135588 [==============================] - 12s 87us/step - loss: 1282.0924 - val_loss: 1154.9337
Epoch 25/30
135588/135588 [==============================] - 11s 83us/step - loss: 1277.2260 - val_loss: 1153.3779
Epoch 26/30
135588/135588 [==============================] - 12s 88us/step - loss: 1275.7087 - val_loss: 1152.5905
Epoch 27/30
135588/135588 [==============================] - 20s 150us/step - loss: 1272.2212 - val_loss: 1153.6475
Epoch 28/30
135588/135588 [==============================] - 20s 148us/step - loss: 1270.67

135588/135588 [==============================] - 21s 154us/step - loss: 1272.4808 - val_loss: 1154.9816
nfold:1,bag:1 1177.51965016
Train on 135588 samples, validate on 33898 samples
Epoch 1/30
135588/135588 [==============================] - 11s 85us/step - loss: 2022.6294 - val_loss: 1248.2605
Epoch 2/30
135588/135588 [==============================] - 11s 83us/step - loss: 1430.1546 - val_loss: 1208.0880
Epoch 3/30
135588/135588 [==============================] - 11s 83us/step - loss: 1401.8442 - val_loss: 1201.0941
Epoch 4/30
135588/135588 [==============================] - 11s 83us/step - loss: 1387.5786 - val_loss: 1191.3741
Epoch 5/30
135588/135588 [==============================] - 11s 83us/step - loss: 1383.0411 - val_loss: 1193.1999
Epoch 6/30
135588/135588 [==============================] - 11s 80us/step - loss: 1369.9199 - val_loss: 1181.8132
Epoch 7/30
135588/135588 [==============================] - 11s 79us/step - loss: 1362.9070 - val_loss: 1176.3032
Epoch 8/30
135588/1

135588/135588 [==============================] - 11s 81us/step - loss: 1391.3166 - val_loss: 1179.7400
Epoch 10/30
135588/135588 [==============================] - 11s 80us/step - loss: 1383.3361 - val_loss: 1180.7782
Epoch 11/30
135588/135588 [==============================] - 11s 82us/step - loss: 1374.1898 - val_loss: 1173.6962
Epoch 12/30
135588/135588 [==============================] - 11s 82us/step - loss: 1369.6746 - val_loss: 1175.0710
Epoch 13/30
135588/135588 [==============================] - 11s 83us/step - loss: 1365.5039 - val_loss: 1174.9235
Epoch 14/30
135588/135588 [==============================] - 11s 81us/step - loss: 1357.8096 - val_loss: 1170.6497
Epoch 15/30
135588/135588 [==============================] - 11s 82us/step - loss: 1349.5745 - val_loss: 1168.3464
Epoch 16/30
135588/135588 [==============================] - 11s 81us/step - loss: 1347.9950 - val_loss: 1168.9486
Epoch 17/30
135588/135588 [==============================] - 11s 82us/step - loss: 1343.0645

...

135588/135588 [==============================] - 12s 85us/step - loss: 1301.4309 - val_loss: 1158.1881
Epoch 16/30
135588/135588 [==============================] - 12s 85us/step - loss: 1301.0248 - val_loss: 1159.5597
Epoch 17/30
135588/135588 [==============================] - 12s 87us/step - loss: 1296.5498 - val_loss: 1160.6282
Epoch 18/30
135588/135588 [==============================] - 11s 84us/step - loss: 1287.8516 - val_loss: 1156.9233
Epoch 19/30
135588/135588 [==============================] - 11s 84us/step - loss: 1288.4059 - val_loss: 1154.9089
Epoch 20/30
135588/135588 [==============================] - 12s 87us/step - loss: 1288.3737 - val_loss: 1155.4500
Epoch 21/30
135588/135588 [==============================] - 11s 84us/step - loss: 1284.8074 - val_loss: 1152.9778
Epoch 22/30
135588/135588 [==============================] - 12s 85us/step - loss: 1281.1596 - val_loss: 1154.9701
Epoch 23/30
135588/135588 [==============================] - 12s 87us/step - loss: 1277.7923

135589/135589 [==============================] - 12s 87us/step - loss: 1318.8721 - val_loss: 1166.4336
Epoch 26/30
135589/135589 [==============================] - 16s 122us/step - loss: 1312.2733 - val_loss: 1163.4217
Epoch 27/30
135589/135589 [==============================] - 29s 213us/step - loss: 1309.7701 - val_loss: 1162.8173
Epoch 28/30
135589/135589 [==============================] - 30s 218us/step - loss: 1307.5007 - val_loss: 1163.5249
Epoch 29/30
135589/135589 [==============================] - 30s 223us/step - loss: 1296.8710 - val_loss: 1162.0324
Epoch 30/30
135589/135589 [==============================] - 30s 219us/step - loss: 1294.8233 - val_loss: 1165.5866
nfold:8,bag:1 1156.6310628
Train on 135589 samples, validate on 33898 samples
Epoch 1/30
135589/135589 [==============================] - 14s 102us/step - loss: 2249.0383 - val_loss: 1297.9463
Epoch 2/30
135589/135589 [==============================] - 12s 90us/step - loss: 1445.5962 - val_loss: 1220.0156
Epoch 3/30

135589/135589 [==============================] - 12s 92us/step - loss: 1308.7301 - val_loss: 1186.5224
Epoch 5/30
135589/135589 [==============================] - 13s 95us/step - loss: 1302.5206 - val_loss: 1182.0422
Epoch 6/30
135589/135589 [==============================] - 12s 92us/step - loss: 1295.3467 - val_loss: 1179.4417
Epoch 7/30
135589/135589 [==============================] - 12s 89us/step - loss: 1292.0860 - val_loss: 1181.0846
Epoch 8/30
135589/135589 [==============================] - 12s 91us/step - loss: 1288.5089 - val_loss: 1175.6624
Epoch 9/30
135589/135589 [==============================] - 12s 88us/step - loss: 1284.1568 - val_loss: 1175.4851
Epoch 10/30
135589/135589 [==============================] - 12s 90us/step - loss: 1281.2952 - val_loss: 1173.1553
Epoch 11/30
135589/135589 [==============================] - 13s 93us/step - loss: 1275.6021 - val_loss: 1170.6424
Epoch 12/30
135589/135589 [==============================] - 13s 93us/step - loss: 1273.4828 - va

135589/135589 [==============================] - 13s 97us/step - loss: 1358.8739 - val_loss: 1170.7539
Epoch 15/30
135589/135589 [==============================] - 13s 93us/step - loss: 1354.2854 - val_loss: 1170.8578
Epoch 16/30
135589/135589 [==============================] - 12s 92us/step - loss: 1351.9035 - val_loss: 1171.0982
Epoch 17/30
135589/135589 [==============================] - 12s 92us/step - loss: 1342.6258 - val_loss: 1166.3252
Epoch 18/30
135589/135589 [==============================] - 13s 93us/step - loss: 1339.2100 - val_loss: 1163.8139
Epoch 19/30
135589/135589 [==============================] - 13s 96us/step - loss: 1330.4638 - val_loss: 1166.2842
Epoch 20/30
135589/135589 [==============================] - 13s 93us/step - loss: 1328.1965 - val_loss: 1166.2774
Epoch 21/30
135589/135589 [==============================] - 13s 97us/step - loss: 1325.1886 - val_loss: 1166.2058
Epoch 22/30
135589/135589 [==============================] - 12s 92us/step - loss: 1320.0434

135589/135589 [==============================] - 13s 96us/step - loss: 1273.0456 - val_loss: 1161.2088
Epoch 25/30
135589/135589 [==============================] - 12s 92us/step - loss: 1271.0892 - val_loss: 1160.2710
Epoch 26/30
135589/135589 [==============================] - 19s 143us/step - loss: 1266.3669 - val_loss: 1164.1971
Epoch 27/30
135589/135589 [==============================] - 33s 242us/step - loss: 1265.9344 - val_loss: 1158.6002
Epoch 28/30
135589/135589 [==============================] - 34s 252us/step - loss: 1263.6676 - val_loss: 1160.4005
Epoch 29/30
135589/135589 [==============================] - 33s 243us/step - loss: 1257.8966 - val_loss: 1159.0719
Epoch 30/30
135589/135589 [==============================] - 33s 241us/step - loss: 1256.2180 - val_loss: 1157.9200
nfold:9,bag:3 1151.07024515
Train on 135589 samples, validate on 33898 samples
Epoch 1/30
135589/135589 [==============================] - 15s 109us/step - loss: 2103.4908 - val_loss: 1302.1991
Epoch 2/

Though this model is not adapted for mere 30 epochs of training, nor for 3-fold CV (I used 5-fold on Kaggle), even though this is a single unbagged model which has been cross-validated on three folds only, we see a very good score:
`CV = 1150` (your score may vary a little).

By the way, this single model, bagged, 5-fold CVed, scored 1116.28 on Kaggle LB.

As we see, this model is considerably better than any other models we had so far. We now take it as the second part of our final ensemble.

In [ ]:
'l3-nn': {
        'predictions': l2_predictions,
        'n_bags': 4,
        'model': Keras(nn_lr, lambda: {'l1': 1e-5, 'l2': 1e-5, 'n_epoch': 30, 'batch_size': 128, 'optimizer': SGD(3e-2, momentum=0.8, nesterov=True, decay=3e-5), 'callbacks': [ExponentialMovingAverage(save_mv_ave_model=False)]}),
    },
     'optimizer': SGD(1e-4, momentum=0.9, nesterov=True, decay=5e-5)
            'optimizer': SGD(1e-5, momentum=0.9, nesterov=True, decay=5e-5)

In [ ]:
{'hidden1_dropout': 0.5368421052631579, 'hidden1_units': 437, 'hidden2_dropout': 0.40000000000000002, 
 'hidden2_units': 182, 'hidden3_dropout': 0.23333333333333334, 'hidden3_units': 73, 'optimizer': 'adadelta',
 'wdecay': 0.0020300000000000001}

In [251]:
def cross_validate_mlp(mlp_func, nfolds=3):
    folds = KFold(len(train_y), n_folds=nfolds, shuffle=True, random_state = 31337)
    val_scores = np.zeros((nfolds,))
    for k,(train_index, test_index) in enumerate(folds):
        xtr = train_x[train_index]
        ytr = train_y[train_index]
        xte = train_x[test_index]
        yte = train_y[test_index]
        mlp = mlp_func()
        early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        fit = mlp.fit(xtr, ytr, validation_split=0.1, batch_size=128, 
                      epochs=10, verbose=0, callbacks=[early_stopping])
        pred = mlp.predict(xte, batch_size=512)
        score = mean_absolute_error(np.exp(yte), np.exp(pred))
        val_scores[k] += score
        print ('Fold {}, MAE: {}'.format(k, score))
    avg_score = np.sum(val_scores) / float(nfolds)
    print ('{}-fold CV score: {}'.format(nfolds, avg_score))
    return avg_score

In [252]:
# VERSION 4. Insights:
# – why not to test 4-layer architectures?
# — we need to introduce new optimizers
# — adding batch normalization (https://arxiv.org/abs/1502.03167)

# Describing the search space
space = {'hidden1_dropout': hp.choice('hidden1_dropout', np.linspace(0.4,0.6,20)),
        'hidden2_dropout': hp.choice('hidden2_dropout', np.linspace(0.2,0.5,10)),
        'hidden3_dropout': hp.choice('hidden3_dropout', np.linspace(0.1,0.5,10)),
         'hidden1_units': hp.choice('hidden1_units', np.linspace(300,550,30,dtype='int32')),
         'hidden2_units': hp.choice('hidden2_units', np.linspace(100,300,30,dtype='int32')),
         'hidden3_units': hp.choice('hidden3_units', np.linspace(20,80,30,dtype='int32')),
         'optimizer': hp.choice('optimizer', ['adadelta','adam']),
         'wdecay':hp.choice('wdecay', np.linspace(0.0001,0.01,1000)),
        }

# Implementing a function to minimize
def hyperopt_search(params):
    print ('Model Testing:', params)
    def mlp_model():
        model = Sequential()
        model.add(Dense(params['hidden1_units'], input_dim=train_x.shape[1], kernel_initializer='he_normal',
                        kernel_regularizer=l2(params['wdecay'])))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['hidden1_dropout']))
        
        model.add(Dense(params['hidden2_units'], kernel_initializer='he_normal',
                        kernel_regularizer=l2(params['wdecay'])))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['hidden2_dropout']))

        model.add(Dense(params['hidden3_units'], kernel_initializer='he_normal',
                        kernel_regularizer=l2(params['wdecay']))) 
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['hidden3_dropout']))
        
        model.add(Dense(1, kernel_initializer='he_normal',kernel_regularizer=l2(params['wdecay'])))
        model.compile(loss=mae_score,metrics=[mae_score], optimizer=params['optimizer'])
        return model
    
    cv_score = cross_validate_mlp(mlp_model)
    return {'loss': cv_score, 'status': STATUS_OK}


trials = Trials()

# UNCOMMENT THE NEXT LINE TO LAUNCH HYPEROPT:
best = fmin(hyperopt_search, space, algo=tpe.suggest, max_evals = 100, trials=trials)

Model Testing: {'hidden1_dropout': 0.59999999999999998, 'hidden1_units': 463, 'hidden2_dropout': 0.46666666666666667, 'hidden2_units': 155, 'hidden3_dropout': 0.18888888888888888, 'hidden3_units': 20, 'optimizer': 'adam', 'wdecay': 0.0012891891891891893}
Fold 0, MAE: 1664.3154518123333
Fold 1, MAE: 1789.6478768193072
Fold 2, MAE: 2230.6679431902985
3-fold CV score: 1894.877090607313
Model Testing: {'hidden1_dropout': 0.50526315789473686, 'hidden1_units': 308, 'hidden2_dropout': 0.20000000000000001, 'hidden2_units': 244, 'hidden3_dropout': 0.41111111111111109, 'hidden3_units': 28, 'optimizer': 'adadelta', 'wdecay': 0.0075522522522522527}
Fold 0, MAE: 2663.589775993532
Fold 1, MAE: 2550.3793884253055
Fold 2, MAE: 2698.309167392798
3-fold CV score: 2637.426110603878
Model Testing: {'hidden1_dropout': 0.59999999999999998, 'hidden1_units': 394, 'hidden2_dropout': 0.23333333333333334, 'hidden2_units': 120, 'hidden3_dropout': 0.18888888888888888, 'hidden3_units': 65, 'optimizer': 'adam', 'wde

KeyboardInterrupt: 